In [1]:
%pip install -U qdrant-client

  Using cached protobuf-5.26.1-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.26.1-cp310-abi3-win_amd64.whl (420 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
opentelemetry-proto 1.23.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.26.1 which is incompatible.


In [2]:
import pandas  as pd

dataset = pd.read_csv('../ozhegov_dataset.csv')
dataset = dataset[dataset['title'].str.len() > 0]
dataset = dataset[dataset['text'].str.len() > 0]
#для lenta - подсовываем даты
#dataset['title'] = dataset['date']
dataset = dataset.sample(n=1000)

dataset.astype({"text": str, "title": str})
dataset.info(show_counts=True)

dataset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 7035 to 6326
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   title       1000 non-null   object
 2   text        1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


,Unnamed: 0,title,text
7035,7035,ЗАДЕЛАТЬСЯ,"Сделаться, стать. Заделался болельщиком. II не..."
18030,18030,ПИОНЕРИЯ,Пионеры (в 3 знач.).
3671,3671,ВЫВАЛЯТЬСЯ,"Валяясь или упав, покрыться чем-н., выпачкатьс..."
6501,6501,ЕЗДКА,Одна из нескольких поездок для привоза или отв...
7678,7678,ЗАСУХА,"Длительное отсутствие дождей, приводящее к выс..."


In [3]:
from qdrant_client import QdrantClient, models as qdrant_models

models = [
    {"model_name":"intfloat/multilingual-e5-large", "size": 1024},
    {"model_name":"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", "size": 768},
    {"model_name":"sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", "size": 384},
    {"model_name":"symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli", "size": 768},
    {"model_name":"sentence-transformers/all-MiniLM-L6-v2", "size": 384},
    {"model_name":"cointegrated/LaBSE-en-ru", "size": 768},
    {"model_name":"sentence-transformers/LaBSE", "size": 768}
]

distances = [
    qdrant_models.Distance.EUCLID,
    qdrant_models.Distance.DOT,
    qdrant_models.Distance.COSINE
]

In [4]:
from qdrant_client import QdrantClient, models as qdrant_models

client = QdrantClient(url="http://localhost:6333")
COLLECTION_NAME="termins"

def create_collection(client, model_name):
    client.set_model(model_name)
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=client.get_fastembed_vector_params()
    )
    add_data()

def add_data():
    ids = list(map(int, dataset.index.values.tolist()))

    client.add(
        collection_name=COLLECTION_NAME,
        ids = ids, documents=dataset["title"].tolist(), batch_size=2, parallel=0
    )


def delete_collection():
    client.delete_collection(collection_name=COLLECTION_NAME)

In [5]:
def query_collection(client, query, max_results, dataframe, model_name):
    results = client.query(
        collection_name=COLLECTION_NAME,
        query_text=query,
        query_filter=None,  # If you don't want any filters for now
        limit=max_results,  # 5 the closest results
    )

    df = pd.DataFrame()
    for res in results:
        df = df._append(pd.DataFrame({
                'id':res.id, 
                'score':res.score,
                'query': query,
                'title': dataframe[dataframe.index == res.id]['title'],
                'content': dataframe[dataframe.index == res.id]['text'],
                'model_name': model_name,
                }))

    # Забираем с максимально высокой оценкой
    df = df[df.score == df.score.max()][:1]
    df['is_found'] = df.apply(lambda row: row.query == row.title, axis=1)
    
    return df

In [6]:
test_dataset = dataset.sample(n=100)
test_dataset.head()
test_results = pd.DataFrame()

In [7]:

for model in models:
    print(f"{model}")
    try:
        delete_collection()
        collection = create_collection(client, model["model_name"])
    except Exception as ex:
        print(f"recreate collection error: {ex}")
        continue
    for title in test_dataset["title"].tolist():
        try:
            test_results = test_results._append(query_collection(
            client,
            query=title,
            max_results=5,
            dataframe=dataset,
            model_name=model["model_name"]))
            print(f"{len(test_results)}")
        except Exception as ex:
            print(f"Test error: {ex}")
            continue
        

test_results.to_csv("results_ozhegov2.csv")

{'model_name': 'intfloat/multilingual-e5-large', 'size': 1024}


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
97
98
99
100
101
{'model_name': 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2', 'size': 768}


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
198
199
200
201
202
{'model_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', 'size': 384}


...

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
400
401
402
403
404
{'model_name': 'cointegrated/LaBSE-en-ru', 'size': 768}
recreate collection error: Unsupported embedding model: cointegrated/LaBSE-en-ru. Supported models: {'BAAI/bge-base-en': (768, <Distance.COSINE: 'Cosine'>), 'BAAI/bge-base-en-v1.5': (768, <Distance.COSINE: 'Cosine'>), 'BAAI/bge-large-en-v1.5': (1024, <Distance.COSINE: 'Cosine'>), 'BAAI/bge-small-en': (384, <Distance.COSINE: 'Cosine'>), 'BAAI/bge-small-en-v1.5': (384, <Distance.COSINE: 'Cosine'>), 'BAAI/bge-small-zh-v1.5': (512, <Distance.COSINE: 'Cosine'>), 'sentence-transformers/all-MiniLM-L6-v2': (384, <Distance.COSINE: 'Cosine'>), 'sen

In [71]:
finally_result = pd.DataFrame()
for model in models:
    df = test_results.loc[test_results['model_name'].str.contains(model["model_name"]) == True]
    finally_result = finally_result._append(pd.DataFrame({
            'founded': [len(df[df['is_found'] == True])],
            'model_name': [model["model_name"]],
            }))
        
finally_result.head(15)
finally_result.to_csv("finally_result_ozhegov2.csv")

In [72]:
finally_result.head(15)


,founded,model_name
0,100,intfloat/multilingual-e5-large
0,100,sentence-transformers/paraphrase-multilingual-...
0,100,sentence-transformers/paraphrase-multilingual-...
0,0,symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli
0,100,sentence-transformers/all-MiniLM-L6-v2
0,0,cointegrated/LaBSE-en-ru
0,0,sentence-transformers/LaBSE
